# Agregación y agrupación

Una pieza esencial en el análisis de grandes datos es un resumen eficiente: calcular agregaciones como ``sum()``, ``mean()``, ``median()``, ``min()``, y ``max()``, en las que un único número da una idea de la naturaleza de un conjunto de datos potencialmente grande.

Volveremos a repasar agregaciones en Pandas, desde operaciones simples similares a las que hemos visto en los arrays de NumPy, y se introducirá ``groupby``.

Por comodidad, utilizaremos la misma función mágica ``display`` que hemos visto en secciones anteriores:

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_15384\2905451983.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Datos de Planetas

Aquí usaremos el conjunto de datos Planetas, disponible a través del [paquete Seaborn](http://seaborn.pydata.org/), que es matplotlib con esteroides.

Proporciona información sobre planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como *planetas extrasolares* o *exoplanetas* para abreviar). Puede descargarse con un simple comando Seaborn:

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [4]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Contiene algunos detalles sobre los más de 1.000 planetas extrasolares descubiertos hasta 2014.

## Agregación simple en Pandas

Anteriormente, hemos explorado algunas de las agregaciones de datos disponibles para arrays NumPy (["Agregaciones: Mín, Máx, y todo lo que hay en medio"](4_Computation-on-arrays-aggregates.ipynb)).
Al igual que con un array unidimensional NumPy, para una ``Serie`` Pandas los agregados devuelven un único valor:

In [5]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [6]:
ser.sum()

2.811925491708157

In [7]:
ser.mean()

0.5623850983416314

Para un ``DataFrame``, por defecto los agregados devuelven resultados dentro de cada columna:

In [8]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [9]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Especificando el argumento ``axis``, puede agregar dentro de cada fila:

In [14]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Las ``Series`` y ``DataFrame`` de Pandas incluyen todos los agregados comunes mencionados en [Operaciones en Pandas](03_Operations-in-Pandas.ipynb); además, hay un método de conveniencia ``describe()`` que calcula varios agregados comunes para cada columna y devuelve el resultado.

Usaremos esto en los datos de Planetas, por ahora eliminando las filas con valores perdidos:

In [16]:
planets.describe(include="all")

,method,number,orbital_period,mass,distance,year
count,1035,1035.000000,992.000000,513.000000,808.000000,1035.000000
unique,10,NaN,NaN,NaN,NaN,NaN
top,Radial Velocity,NaN,NaN,NaN,NaN,NaN
freq,553,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,NaN,1.240976,26014.728304,3.818617,733.116493,3.972567
min,NaN,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,NaN,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,NaN,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,NaN,2.000000,526.005000,3.040000,178.500000,2012.000000


In [17]:
planets['method'].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

In [15]:
planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [14]:
# Cuidado que dropna puede ser muy "agresivo". Más del 50%!!!
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


La siguiente tabla resume algunas otras agregaciones incorporadas en Pandas:

| Agregacion               | Descripcion                     |
|--------------------------|---------------------------------|
| ``count()``              | Número total de artículos       |
| ``first()``, ``last()``  | Primer y último punto           |
| ``mean()``, ``median()`` | Media y mediana                 |
| ``min()``, ``max()``     | Mínimo y máximo                 |
| ``std()``, ``var()``     | Desviación estandar y varianza  |
| ``mad()``                | Desviación media absoluta       |
| ``prod()``               | Producto de todos los artículos |
| ``sum()``                | Suma de todas las partidas      |

Todos estos son métodos de los objetos ``DataFrame`` y ``Series``.

Esta puede ser una forma útil de empezar a comprender las propiedades generales de un conjunto de datos.
Por ejemplo, en la columna ``year`` vemos que, aunque ya en 1989 se descubrieron exoplanetas, la mitad de los conocidos no se descubrieron hasta 2010 o después.
Esto se debe en gran parte a la misión *Kepler*, un telescopio espacial diseñado específicamente para encontrar planetas eclipsantes alrededor de otras estrellas.

Sin embargo, para profundizar en los datos, los agregados simples no suelen ser suficientes.
El siguiente nivel de integración de datos es la operación ``groupby``, que permite calcular rápida y eficazmente agregados sobre subconjuntos de datos.

## GroupBy: Dividir, Aplicar, Combinar

Las agregaciones simples pueden darle una idea de su conjunto de datos, pero a menudo preferiríamos agregar condicionalmente en alguna etiqueta o índice: esto se implementa en la llamada operación ``groupby``.

El nombre "group by" procede de un comando del lenguaje de bases de datos SQL, pero quizá sea más ilustrativo pensar en él en los términos acuñados por Hadley Wickham, famoso por Rstats: *dividir, aplicar, combinar*.

### Split, apply, combine

Un ejemplo canónico de esta operación dividir-aplicar-combinar, donde el "aplicar" es una agregación sumatoria.

Esto aclara lo que consigue el ``groupby``:

- El paso *split* consiste en dividir y agrupar un ``DataFrame`` en función del valor de la clave especificada.
- El paso *apply* implica calcular alguna función, normalmente un agregado, transformación o filtrado, dentro de los grupos individuales.
- El paso *combinar* combina los resultados de estas operaciones en una matriz de salida.

Aunque esto podría hacerse manualmente utilizando alguna combinación de los comandos de enmascaramiento, agregación y fusión descritos anteriormente, es importante tener en cuenta que *las divisiones intermedias no necesitan instanciarse explícitamente*. En su lugar, ``GroupBy`` puede (a menudo) hacer esto en una sola pasada sobre los datos, actualizando la suma, media, recuento, min, u otro agregado para cada grupo a lo largo del camino.
El poder del ``GroupBy`` es que abstrae estos pasos: el usuario no necesita pensar en *cómo* se realiza el cálculo bajo el capó, sino que piensa en la *operación en su conjunto*.

Como ejemplo concreto, echemos un vistazo al uso de Pandas para el cálculo mostrado en este diagrama.
Empezaremos creando el ``DataFrame`` de entrada:

In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame({'department': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'VV': range(6)})
df

,department,VV
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de dividir-aplicar-combinar puede calcularse con el método ``groupby()`` de ``DataFrame``s, pasando el nombre de la columna clave deseada:

In [32]:
df.groupby('department').sum().mean()

VV    5.0
dtype: float64

In [33]:
df['VV'].mean()

2.5

Observa que lo que se devuelve no es un conjunto de ``DataFrame``s, sino un objeto ``DataFrameGroupBy``.
Este objeto es donde está la magia: se puede pensar en él como una vista especial del ``DataFrame``, que está preparado para profundizar en los grupos, pero no hace ningún cálculo real hasta que se aplica la agregación.
Este enfoque de "evaluación perezosa" significa que los agregados comunes pueden implementarse muy eficientemente de un modo casi transparente para el usuario.

Para producir un resultado, podemos aplicar un agregado a este objeto ``DataFrameGroupBy``, que realizará los pasos apropiados de aplicar/combinar para producir el resultado deseado:

In [18]:
df_grouped = df.groupby('department').max()
df_grouped

,VV
department,
A,3
B,4
C,5


El método ``sum()`` es sólo una posibilidad aquí; puedes aplicar virtualmente cualquier función de agregación común de Pandas o NumPy, así como virtualmente cualquier operación válida de ``DataFrame``, como veremos en la siguiente discusión.

### El objeto GroupBy

El objeto ``GroupBy`` es una abstracción muy flexible.
En muchos sentidos, puedes simplemente tratarlo como si fuera una colección de ``DataFrame``s, y hace las cosas difíciles bajo el capó. Veamos algunos ejemplos utilizando los datos de Los Planetas.

Quizás las operaciones más importantes disponibles en un ``GroupBy`` son *agregar*, *filtrar*, *transformar* y *aplicar*.

#### Indexación de columnas

El objeto ``GroupBy`` soporta la indexación por columnas de la misma forma que el ``DataFrame``, y devuelve un objeto ``GroupBy`` modificado.
Por ejemplo

In [19]:
planets.groupby('method')

In [34]:
planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [43]:
planets.groupby(['method'])[['number', 'mass']].sum()

,number,mass
method,,
Astrometry,2,0.00000
Eclipse Timing Variations,15,10.25000
Imaging,50,0.00000
Microlensing,27,0.00000
Orbital Brightness Modulation,5,0.00000
Pulsar Timing,11,0.00000
Pulsation Timing Variations,1,0.00000
Radial Velocity,952,1341.65638
Transit,776,1.47000


In [20]:
planets.groupby('method')['orbital_period']

Aquí hemos seleccionado un grupo ``Series`` particular del grupo original ``DataFrame`` por referencia a su nombre de columna.

Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamamos a algún agregado en el objeto:

In [21]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [22]:
len(planets['method'].unique())

10

In [46]:
# Son dobles corchetes
planets.groupby('method')[['orbital_period']].mean()

,orbital_period
method,
Astrometry,631.180000
Eclipse Timing Variations,4751.644444
Imaging,118247.737500
Microlensing,3153.571429
Orbital Brightness Modulation,0.709307
Pulsar Timing,7343.021201
Pulsation Timing Variations,1170.000000
Radial Velocity,823.354680
Transit,21.102073


In [48]:
# Sanity Check
planets[planets['method']=='Transit'][['orbital_period']].mean()

orbital_period    21.102073
dtype: float64

Esto da una idea de la escala general de periodos orbitales (en días) a la que es sensible cada método.

#### Iteración sobre grupos

El objeto ``GroupBy`` permite la iteración directa sobre los grupos, devolviendo cada grupo como una ``Series`` o ``DataFrame``:

In [49]:
## No demasiado relevante
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


Esto puede ser útil para hacer ciertas cosas manualmente, aunque a menudo es mucho más rápido utilizar la funcionalidad incorporada ``apply``, que discutiremos momentáneamente.

#### Métodos de envío

A través de la magia de las clases de Python, cualquier método no implementado explícitamente por el objeto ``GroupBy`` será pasado y llamado en los grupos, ya sean objetos ``DataFrame`` o ``Series``.
Por ejemplo, puedes utilizar el método ``describe()`` de ``DataFrame`` para realizar un conjunto de agregaciones que describan cada grupo en los datos:

In [50]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.0
       Eclipse Timing Variations           9.0
       Imaging                            38.0
       Microlensing                       23.0
       Orbital Brightness Modulation       3.0
                                         ...  
max    Pulsar Timing                    2011.0
       Pulsation Timing Variations      2007.0
       Radial Velocity                  2014.0
       Transit                          2014.0
       Transit Timing Variations        2014.0
Length: 80, dtype: float64

Observar esta tabla nos ayuda a entender mejor los datos: por ejemplo, la gran mayoría de los planetas se han descubierto por los métodos de Velocidad Radial y Tránsito, aunque este último no se hizo común (debido a nuevos telescopios más precisos) hasta la última década.
Los métodos más recientes parecen ser el de la Variación Temporal del Tránsito y el de la Modulación del Brillo Orbital, que no se utilizaron para descubrir un nuevo planeta hasta 2011.

Este es solo un ejemplo de la utilidad de los métodos de envío.
Fíjate en que se aplican *a cada grupo individual*, y los resultados se combinan dentro de ``GroupBy`` y se devuelven.
De nuevo, cualquier método válido de ``DataFrame`` o ``Series`` puede utilizarse en el objeto ``GroupBy`` correspondiente, lo que permite realizar operaciones muy flexibles y potentes.

### Agregar, filtrar, transformar, aplicar

La discusión anterior se centró en la agregación para la operación combinar, pero hay más opciones disponibles.
En particular, los objetos ``GroupBy`` tienen métodos ``aggregate()``, ``filter()``, ``transform()``, y ``apply()`` que implementan eficientemente una variedad de operaciones útiles antes de combinar los datos agrupados.

Para el propósito de las siguientes subsecciones, utilizaremos este ``DataFrame``:

In [52]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Agregación

Ahora estamos familiarizados con las agregaciones ``GroupBy`` con ``sum()``, ``median()``, y similares, pero el método ``aggregate()`` permite aún más flexibilidad.
Puede tomar una cadena, una función o una lista de ellas y calcular todos los agregados a la vez.
He aquí un ejemplo rápido que combina todos estos métodos:

In [53]:
df.groupby('key').aggregate(['min', 'median', 'max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Otro patrón útil es pasar un diccionario que asigne los nombres de las columnas a las operaciones que deben aplicarse a esa columna:

In [58]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [59]:
df.groupby('key').aggregate({'data1': ['min', 'std'],
                             'data2': ['max', "sum"]})

data1          data2    
      min      std   max sum
key                         
A       0  2.12132     5   8
B       1  2.12132     7   7
C       2  2.12132     9  12

#### Filtrado

Una operación de filtrado permite descartar datos en función de las propiedades del grupo.
Por ejemplo, podríamos querer mantener todos los grupos en los que la desviación estándar es mayor que algún valor crítico:

In [80]:
# Supongamos que son las notas de unos alumnos y apruebas si y solo si has sacado más de un 3 en TODOS los examenes 
def filter_func(x):
    return x['examen'].mean() >= 5
notas = pd.DataFrame({'estudiante': ['Juan', 'Ana', 'Ester', 'Juan', 'Ana', 'Ester'],
                               'trimestre': [1,1,1,2,2,2],
                   'practicas': [6,7,2,4,5,7],
                   'actitud': [True, False, True, True,False,False],
                   'examen': [6,7,8,3,5,2]},
          
                   columns = ['estudiante', 'trimestre', 'practicas', 'actitud', 'examen'])

In [81]:
notas

,estudiante,trimestre,practicas,actitud,examen
0,Juan,1,6,True,6
1,Ana,1,7,False,7
2,Ester,1,2,True,8
3,Juan,2,4,True,3
4,Ana,2,5,False,5
5,Ester,2,7,False,2


In [85]:
def filter_func(x):
    x['practicas'] = x['practicas'] 
    return x['practicas'].mean() >= 5
notas.groupby('estudiante').filter(filter_func)

,estudiante,trimestre,practicas,actitud,examen
0,Juan,1,6,True,6
1,Ana,1,7,False,7
3,Juan,2,4,True,3
4,Ana,2,5,False,5


In [86]:
display('notas', "notas.groupby('estudiante').min()", "notas.groupby('estudiante').filter(filter_func)")

notas
  estudiante  trimestre  practicas  actitud  examen
0       Juan          1          6     True       6
1        Ana          1          7    False       7
2      Ester          1          2     True       8
3       Juan          2          4     True       3
4        Ana          2          5    False       5
5      Ester          2          7    False       2

notas.groupby('estudiante').min()
            trimestre  practicas  actitud  examen
estudiante                                       
Ana                 1          5    False       5
Ester               1          2    False       2
Juan                1          4     True       3

notas.groupby('estudiante').filter(filter_func)
  estudiante  trimestre  practicas  actitud  examen
0       Juan          1          6     True       6
1        Ana          1          7    False       7
3       Juan          2          4     True       3
4        Ana          2          5    False       5

In [91]:
notas[notas['examen'] > 3]

,estudiante,trimestre,practicas,actitud,examen
0,Juan,1,6,True,6
1,Ana,1,7,False,7
2,Ester,1,2,True,8
4,Ana,2,5,False,5


In [100]:
# Supongamos que son las notas de unos alumnos y apruebas si y solo si has sacado más de un 3 en TODOS los examenes 
def filter_func(x):
    return x['examen'].min() > 3
notas = pd.DataFrame({'estudiante': ['Juan', 'Ana', 'Ester', 'Juan', 'Ana', 'Ester'],
                   'practicas': [6,7,2,4,5,7],
                   'examen': [6,7,8,3,5,2]},
                   columns = ['estudiante', 'practicas', 'examen'])

In [93]:
notas

,estudiante,practicas,examen
0,Juan,6,6
1,Ana,7,7
2,Ester,2,8
3,Juan,4,3
4,Ana,5,5
5,Ester,7,2


In [95]:
def filter_func(x):
    return x['examen'].min() > 3
notas.groupby('estudiante').filter(filter_func)

,estudiante,practicas,examen
1,Ana,7,7
4,Ana,5,5


La función de filtrado debe devolver un valor booleano que especifique si el grupo pasa el filtrado. En este caso, como el grupo A no tiene una desviación típica superior a 4, se elimina del resultado.

#### Transformación

Mientras que la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinar.

Para tal transformación, la salida tiene la misma forma que la entrada.

Un ejemplo común es centrar los datos restando la media del grupo:

In [106]:
notas

,estudiante,practicas,examen
0,Juan,6,6
1,Ana,7,7
2,Ester,2,8
3,Juan,4,3
4,Ana,5,5
5,Ester,7,2


In [107]:

notas.groupby('estudiante').transform(lambda x: x - x.mean())

,practicas,examen
0,1.0,1.5
1,1.0,1.0
2,-2.5,3.0
3,-1.0,-1.5
4,-1.0,-1.0
5,2.5,-3.0


In [104]:
notas.groupby('estudiante').std()

,practicas,examen
estudiante,,
Ana,1.414214,1.414214
Ester,3.535534,4.242641
Juan,1.414214,2.121320


In [105]:
pd.merge(notas,notas.groupby('estudiante').transform(lambda x: x - x.mean()), left_index=True, right_index=True, suffixes=("_nota", "_des"))

,estudiante,practicas_nota,examen_nota,practicas_des,examen_des
0,Juan,6,6,1.0,1.5
1,Ana,7,7,1.0,1.0
2,Ester,2,8,-2.5,3.0
3,Juan,4,3,-1.0,-1.5
4,Ana,5,5,-1.0,-1.0
5,Ester,7,2,2.5,-3.0


In [89]:
notas.groupby("estudiante").mean()

,practicas,examen
estudiante,,
Ana,6.0,6.0
Ester,4.5,5.0
Juan,5.0,4.5


In [90]:
notas['examen_transformed'] = df['examen'] - df['examen'].mean()
notas

,estudiante,practicas,examen,examen_transformed
0,Juan,6,6,0.833333
1,Ana,7,7,1.833333
2,Ester,2,8,2.833333
3,Juan,4,3,-2.166667
4,Ana,5,5,-0.166667
5,Ester,7,2,-3.166667


#### El método apply()

El método ``apply()`` permite aplicar una función arbitraria a los resultados del grupo.

La función debe tomar un ``DataFrame``, y devolver un objeto Pandas (por ejemplo, ``DataFrame``, ``Series``) o un escalar; la operación de combinación se adaptará al tipo de salida devuelta.

Por ejemplo, aquí hay una ``apply()`` que normaliza la primera columna por la suma de la segunda:

In [109]:
notas = pd.DataFrame({'estudiante': ['Juan', 'Ana', 'Ester', 'Juan', 'Ana', 'Ester'],
                   'practicas': [6,7,2,4,5,7],
                   'examen': [6,7,8,3,5,2]},
                   columns = ['estudiante', 'practicas', 'examen'])
def norm_by_data2(x):
    # x es un DataFrame de valores de grupo
    x['final'] = (0.8*x['examen'].sum() + 0.2*x['practicas'].sum())/len(x) 
    return x

display('notas', "notas.groupby('estudiante').apply(norm_by_data2)")

<string>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<string>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


notas
  estudiante  practicas  examen
0       Juan          6       6
1        Ana          7       7
2      Ester          2       8
3       Juan          4       3
4        Ana          5       5
5      Ester          7       2

notas.groupby('estudiante').apply(norm_by_data2)
             estudiante  practicas  examen  final
estudiante                                       
Ana        1        Ana          7       7    6.0
           4        Ana          5       5    6.0
Ester      2      Ester          2       8    4.9
           5      Ester          7       2    4.9
Juan       0       Juan          6       6    4.6
           3       Juan          4       3    4.6

In [112]:
(12*0.8+12*0.2)/2

6.000000000000001

In [113]:
(10*0.8+9*0.2)/2

4.9

In [110]:
notas.groupby('estudiante').apply(norm_by_data2)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_15384\1689086174.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  notas.groupby('estudiante').apply(norm_by_data2)


estudiante  practicas  examen  final
estudiante                                       
Ana        1        Ana          7       7    6.0
           4        Ana          5       5    6.0
Ester      2      Ester          2       8    4.9
           5      Ester          7       2    4.9
Juan       0       Juan          6       6    4.6
           3       Juan          4       3    4.6

In [114]:
# Diferencia: apply va fila a fila, transform grupo a grupo
notas.groupby('estudiante').transform(norm_by_data2)

KeyError: 'examen'

La función ``apply()`` dentro de un ``GroupBy`` es bastante flexible: el único criterio es que la función toma un ``DataFrame`` y devuelve un objeto Pandas o un escalar; ¡lo que hagas en medio depende de ti!

### Especificar la clave de división

En los ejemplos sencillos presentados anteriormente, dividimos el ``DataFrame`` en un único nombre de columna.

Esta es sólo una de las muchas opciones por las que los grupos se pueden definir, y vamos a ir a través de algunas otras opciones para la especificación de grupo aquí.

#### Una lista, matriz, serie o índice que proporcione las claves de agrupación.

La clave puede ser cualquier serie o lista cuya longitud coincida con la del ``DataFrame``. Por ejemplo:

In [116]:
#No estudiar
L = [0, 1, 2, 3, 4, 5]
display('notas', 'notas.groupby(L).sum()')

,estudiante,practicas,examen
0,Juan,6,6
1,Ana,7,7
2,Ester,2,8
3,Juan,4,3
4,Ana,5,5
5,Ester,7,2
,estudiante,practicas,examen
0,Juan,6,6
1,Ana,7,7
2,Ester,2,8


In [117]:
# Raro, pero puede ser útil en algún caso
L = [0, 1, 0, 0, 1, 0]
display('notas', 'notas.groupby(L).sum()')

,estudiante,practicas,examen
0,Juan,6,6
1,Ana,7,7
2,Ester,2,8
3,Juan,4,3
4,Ana,5,5
5,Ester,7,2
,estudiante,practicas,examen
0,JuanEsterJuanEster,19,19
1,AnaAna,12,12


Por supuesto, esto significa que hay otra forma más verbosa de realizar el ``df.groupby('clave')`` de antes:

In [118]:
notas = pd.DataFrame({'estudiante': ['Juan', 'Ana', 'Ester', 'Juan', 'Ana', 'Ester','Juan', 'Ana', 'Ester', 'Juan', 'Ana', 'Ester'],
                   'practicas': [6,7,2,4,5,7,8,3,5,1,7,7],
                   'examen': [6,7,8,3,5,2, 6,5,6,2,4,1]},
                   columns = ['estudiante', 'practicas', 'examen'])
notas['trimestre'] = [0,0,0,0,0,0,1,1,1,1,1,1]
notas.groupby(['estudiante', 'trimestre']).mean()

practicas  examen
estudiante trimestre                   
Ana        0                6.0     6.0
           1                5.0     4.5
Ester      0                4.5     5.0
           1                6.0     3.5
Juan       0                5.0     4.5
           1                4.5     4.0

#### Un diccionario o serie que asigna un índice a un grupo

Otro método consiste en proporcionar un diccionario que asigne los valores del índice a las claves del grupo:

In [119]:
notas2 = notas.set_index('estudiante')
mapping = {'Ana': 'Beca', 'Ester': 'No Beca', 'Juan': 'Beca'}
display('notas2', 'notas2.groupby(mapping).median()')

,practicas,examen,trimestre
estudiante,,,
Juan,6,6,0
Ana,7,7,0
Ester,2,8,0
Juan,4,3,0
Ana,5,5,0
Ester,7,2,0
Juan,8,6,1
Ana,3,5,1
Ester,5,6,1


In [121]:
notas2[['practicas', 'examen']].groupby(mapping).median()

,practicas,examen
estudiante,,
Beca,5.5,5.0
No Beca,6.0,4.0


In [122]:
# Esto ya no funciona
notas2['practicas', 'examen']

KeyError: ('practicas', 'examen')